In [2]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.optim as optim
import os
import shutil
import random

In [ ]:
!unzip /kaggle/input/dogs-vs-cats/train.zip

In [4]:
def split_dataset(input_folder, train_folder, test_folder, train_ratio = 0.8):
    if not os.path.exists(train_folder):
        os.makedirs(train_folder)
    if not os.path.exists(test_folder):
        os.makedirs(test_folder)
    
    image_files = []
    for image_path in os.listdir(input_folder):
        image_files.append(image_path)
    
    random.shuffle(image_files)
    
    total_images = len(image_files)
    num_train = int(total_images * train_ratio)
    num_test = total_images - num_train
    
    for img_file in image_files[: num_train]:
        src_path = os.path.join(input_folder, img_file)
        dst_path = os.path.join(train_folder, img_file)
        shutil.copy(src_path, dst_path)
        
    for img_file in image_files[num_train:]:
        src_path = os.path.join(input_folder, img_file)
        dst_path = os.path.join(test_folder, img_file)
        shutil.copy(src_path, dst_path)
    
input_folder = '/kaggle/working/train'
train_folder = '/kaggle/working/train_data'
test_folder = '/kaggle/working/test_data'

split_dataset(input_folder, train_folder, test_folder)
    
    

In [5]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [7]:
train_dataset = datasets.ImageFolder(root = "/kaggle/working/train_data", transform = transform)
test_dataset = datasets.ImageFolder(root = "/kaggle/working/test_data", transform = transform)


FileNotFoundError: Couldn't find any class folder in /kaggle/working/train_data.

In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 32, shuffle = False)


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 53 * 53, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,2)
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = x.view(-1, 16 * 53 * 53)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        
        return x

In [ ]:
model = CNN()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [ ]:
for epoch in range(10):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = loss_function(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 100 == 0:
            print("Epoch [{}/{}]. Step [{}/{}]. Loss: {:.4f}".format(epoch + 1, 10, i+1, len(train_loader), loss.item()))